In [1]:
import numpy as np
import pandas as pd
import data_process as dp
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('data.csv')
DOdata=data[['6-1生物池DO1','6-1生物池DO2','6-1生物池DO3','6-1生物池DO4']].mean(axis=1)
data['生物池DO']=DOdata
DOdata=data[['6-1好氧段曝气量1','6-1好氧段曝气量2','6-1好氧段曝气量3','6-1好氧段曝气量4']].mean(axis=1)
data['好氧段曝气量']=DOdata
print(data.columns)

Index(['Unnamed: 0', '日期', '出水COD', '出水TN', '出水TP', '出水氨氮', '出水NO3-',
       '6-1号生物池好氧段MLSS', '6-1号生物池缺氧段MLSS', '6-1号生物池厌氧段MLSS', '6-1号生物池缺氧段OPR',
       '6-1号生物池厌氧段OPR', '6-1生物池DO1', '6-1生物池DO2', '6-1生物池DO3', '6-1生物池DO4',
       '6-1号生物池好氧段NO3', '进水COD', '进水PH', '进水温度', '进水氨氮', '进水TP', '进水TN',
       '进水SS', '1号生物反应池瞬时流量', '6-1好氧段曝气量1', '6-1好氧段曝气量2', '6-1好氧段曝气量3',
       '6-1好氧段曝气量4', '7-1外回流流量1', '内回流流量1', '生物池DO', '好氧段曝气量'],
      dtype='object')


In [3]:
sdata=data[['6-1号生物池好氧段MLSS', '6-1号生物池缺氧段MLSS', '6-1号生物池厌氧段MLSS', 
            '6-1号生物池缺氧段OPR','6-1号生物池厌氧段OPR','生物池DO','1号生物反应池瞬时流量']]
cdata=data[['好氧段曝气量','7-1外回流流量1', '内回流流量1']]
idata=data[['进水COD', '进水PH', '进水温度', '进水氨氮', '进水TP', '进水TN', '进水SS']]
odata=data[['出水COD', '出水TN', '出水TP', '出水氨氮', '出水NO3-']]

# Reconstruct training and test data

In [4]:
t_sDATA,smaxd,smind=dp.normlize_F(sdata.values)
t_cDATA,cmaxd,cmind=dp.normlize_F(cdata.values)
t_iDATA,imaxd,imind=dp.normlize_F(idata.values)
t_oDATA,omaxd,omind=dp.normlize_F(odata.values)

In [5]:
np.save('./tem_data/sDatamax.npy',smaxd)
np.save('./tem_data/sDatamin.npy',smind)
np.save('./tem_data/cDatamax.npy',cmaxd)
np.save('./tem_data/cDatamin.npy',cmind)
np.save('./tem_data/iDatamax.npy',imaxd)
np.save('./tem_data/iDatamin.npy',imind)
np.save('./tem_data/oDatamax.npy',omaxd)
np.save('./tem_data/oDatamin.npy',omind)

In [6]:
HRT=84 # time=7 hour, equal to 84 steps
outstep=6 # predict 30 min, equal to 6 steps
n=data.shape[0]
trainingset=[np.random.randint(HRT+10,n-outstep,1)[0] for _ in range(1000)]
valset=[np.random.randint(HRT+10,n-outstep,1)[0] for _ in range(500)]
np.save('./tem_data/v3/trainingset.npy',trainingset)
np.save('./tem_data/v3/valset.npy',valset)

In [7]:
sTrain,cTrain,iTrain,oTrain,oTrain_out=[],[],[],[],[]
sVal,cVal,iVal,oVal,oVal_out=[],[],[],[],[]
sTest,cTest,iTest,oTest,oTest_out=[],[],[],[],[]

for i in range(HRT+10,n-outstep):

    if i in trainingset:
        sTrain.append(t_sDATA[i-HRT:i,:].reshape((1,-1))[0].tolist())
        cTrain.append(t_cDATA[i-HRT:i,:].reshape((1,-1))[0].tolist())
        iTrain.append(t_iDATA[i-HRT:i,:].reshape((1,-1))[0].tolist())
        oTrain.append(t_oDATA[i,:].tolist())
        oTrain_out.append(t_oDATA[i+outstep,:].tolist())
    elif i in valset:
        sVal.append(t_sDATA[i-HRT:i,:].reshape((1,-1))[0].tolist())
        cVal.append(t_cDATA[i-HRT:i,:].reshape((1,-1))[0].tolist())
        iVal.append(t_iDATA[i-HRT:i,:].reshape((1,-1))[0].tolist())
        oVal.append(t_oDATA[i,:].tolist())
        oVal_out.append(t_oDATA[i+outstep,:].tolist())
    sTest.append(t_sDATA[i-HRT:i,:].reshape((1,-1))[0].tolist())
    cTest.append(t_cDATA[i-HRT:i,:].reshape((1,-1))[0].tolist())
    iTest.append(t_iDATA[i-HRT:i,:].reshape((1,-1))[0].tolist())
    oTest.append(t_oDATA[i,:].tolist())
    oTest_out.append(t_oDATA[i+outstep,:].tolist())

# Training set
sTrain=np.array(sTrain)
cTrain=np.array(cTrain)
iTrain=np.array(iTrain)
oTrain=np.array(oTrain)
oTrain_out=np.array(oTrain_out)

# Validation set
sVal=np.array(sVal)
cVal=np.array(cVal)
iVal=np.array(iVal)
oVal=np.array(oVal)
oVal_out=np.array(oVal_out)

# Test set
sTest=np.array(sTest)
cTest=np.array(cTest)
iTest=np.array(iTest)
oTest=np.array(oTest)
oTest_out=np.array(oTest_out)

In [8]:
# save data
np.save('./tem_data/v3/sTrain.npy',sTrain)
np.save('./tem_data/v3/cTrain.npy',cTrain)
np.save('./tem_data/v3/iTrain.npy',iTrain)
np.save('./tem_data/v3/oTrain.npy',oTrain)
np.save('./tem_data/v3/oTrain_out.npy',oTrain_out)

np.save('./tem_data/v3/sVal.npy',sVal)
np.save('./tem_data/v3/cVal.npy',cVal)
np.save('./tem_data/v3/iVal.npy',iVal)
np.save('./tem_data/v3/oVal.npy',oVal)
np.save('./tem_data/v3/oVal_out.npy',oVal_out)

np.save('./tem_data/v3/sTest.npy',sTest)
np.save('./tem_data/v3/cTest.npy',cTest)
np.save('./tem_data/v3/iTest.npy',iTest)
np.save('./tem_data/v3/oTest.npy',oTest)
np.save('./tem_data/v3/oTest_out.npy',oTest_out)